# Solução dos problemas propostos n aula 02 - Interpolação trigonométrica


In [ ]:
using PyPlot
using FFTW

# Ortogonalidade discreta
Lembrando que 
\begin{align}
    2\cos\theta\cos\phi &= \cos(\theta-\phi) + \cos(\theta+\phi)\\
    2\cos\theta\sin\phi &= \sin(\theta+\phi) - \sin(\theta-\phi)\\
\end{align}

Chega-se à relação de ortogonalidade desejada:
$$
\sum_{i=0}^Q \phi_n(x_i)\cdot \phi_k(x_i)=0 \qquad k\ne n, N\: \text{não é múltiplo de}\: Q/2
$$

In [ ]:
function fourierseries(x, L0, a0, a, b)
    y = a0/2
    N = length(a)
    for i = 1:N
        y += a[i] * cos(2π*i*x/L0)
        y += b[i] * sin(2π*i*x/L0)
    end
    return y
end

function fourier_coeffs(N, x, y, L0)
    Q = length(x)
    a0 = 2/Q * sum(y)
    an = 2/Q * [sum(y .* cos.(2π .* n .* x ./ L0)) for n = 1:N]
    bn = 2/Q * [sum(y .* sin.(2π .* n .* x ./ L0)) for n = 1:N]
    return a0, an, bn
end

    
    
    

In [ ]:
#nfftutil(N) = (N % 2 == 0) ? (div(N,2) + 1) : div(N+1,2)
nfftutil(N) = div(N,2) + 1


## Interpolação trigonométrica

In [ ]:
struct TrigInterp
    Q::Int
    dx::Float64
    c::Vector{ComplexF64}
end
TrigInterp(x::Vector{Float64}, dx) = TrigInterp(length(x), dx, rfft(x))

function interp(f::TrigInterp, x)
    y = real(f.c[1])/2
    Q = f.Q
    P = Q * f.dx
    N = length(f.c)
    for k in 1:N-1
        c = f.c[k+1]
        y += real(c) * cos(2π*k*x/P) - imag(c) * sin(2π*k*x)
    end
    
    return 2y/Q
end

(f::TrigInterp)(x) = interp(f, x)

# Problemas


## Problema 1

Seja a seguinte função definida no domínio $-1 < x \le 1$:

$$
y(x) = \left\{\begin{matrix} -1 & x < 0 \\ 1 & x > 0\\ 0 & x=0\end{matrix}\right.
$$

 1. Ache os coeficientes $a_n$ e $b_n$ da série de Fourier
 2. Plote a série de Fourier para diferentes números de termos da série de Fourier. Percebe algo estranho?
 3. Qual a relação desta função com a função $y = |x|$ que estudamos anteriormente?
 4. Escolha diferentes números de pontos e interpole esta função nestes pontos

### Solução

#### Ache os coeficientes $𝑎_n$ e $𝑏_𝑛$ da série de Fourier

Esta é uma função ímpar, portanto os coeficientes $a_n$ são nulos!

$$
b_n = \frac{2}{L_0}\int_{-L_0/2}^{L_0/2} y(x) \cdot \sin\left(\frac{2\pi n x}{L_0}\right)\:dx = 2\int_0^1 \sin\left(\pi n x\right)\:dx = \frac{2}{n\pi}\left(1 - \cos n\pi\right) = \frac{2}{n\pi}\left[1 - (-1)^n\right]
$$


In [ ]:
x = range(-1.0, 1.0, length=201);
y0 = 1.0 .* sign.(x)
bn = [2/(π*n) * (1.0 - cos(π*n)) for n in 1:500];

In [ ]:
sinseries(x, b, N, L0) = sum(b[n]*sin(2π*n*x/L0) for n in 1:N);

#### Plote a série de Fourier para diferentes números de termos da série de Fourier

In [ ]:
plot(x, y0, "k:")
for n in 1:2:15
    plot(x, sinseries.(x, Ref(bn), n, 2.0), label="n = $n")
end
legend()

In [ ]:
x1 = range(-1, 1, length=1001)
plot(x1, sinseries.(x1, Ref(bn), 30, 2.0))

#### Qual a relação desta função com a função $y = |x|$ que estudamos anteriormente?

Esta função é a derivada da função $y = |x|$!

E a série de $dy/dx$ é formada pelas derivadas da série original!

Existem condições específicas onde se pode derivar a série termo a termo. Mas se as funções forem suaves, derive e integre a vontade! Funciona até para algumas descontinuidades.




#### Escolha diferentes números de pontos e interpole esta função nestes pontos

In [ ]:
qpoints(Q, a=-1.0, b=1.0) = range(a, b, length=Q+1)[1:Q]


In [ ]:
Q = 8
x = qpoints(Q, 0.0, 1.0)
dx  = x[2]-x[1]
f = 1
y = cos.(2π*f.*x) .+ 2*sin.(2π*f.*x);
Y = fft(y)./Q
freq = fftfreq(Q, 1/dx)

hcat(freq, round.(real.(Y), digits=2), round.(imag.(Y), digits=2))



In [ ]:
Q = 8
x = qpoints(Q, 0.0, 1.0)
dx  = x[2]-x[1]
f = 1
y = cos.(2π*f.*x) .+ 2*sin.(2π*f.*x);
Y = rfft(y)./Q
freq = rfftfreq(Q, 1/dx)

hcat(freq, round.(real.(Y), digits=2), round.(imag.(Y), digits=2))


In [ ]:
## Resample

function myresample(y, Qout)
    Qin = length(y)
    Nin = Qin ÷ 2 + 1
    Nout = Qout ÷ 2 + 1
    
    Yin = rfft(y)
    Yout = zeros(ComplexF64, Nout)
    nmin = min(Nout, Nin)
    for i in 1:nmin
        Yout[i] = Qout/Qin * Yin[i]
    end
    
    
    if Qout > Qin  # interpolação
        if iseven(Qin)
            Yout[Nin] /= 2
        end
    elseif Qout < Qin
        if iseven(Qin)
            Yout[Nout] = 2*real(Yout[Nout])
        end
    end
    
    return irfft(Yout, Qout)
end

In [ ]:
Q0 = 16
x0 = range(-1, 1, length=Q0+1)[1:Q0]
y0 = 1.0 * sign.(x0);

In [ ]:
Q1 =  16
x1 = qpoints(Q1, -1, 1)
y1 = myresample(y0, Q1)
plot(x1, y1)
plot(x0, y0, "r.")


## Problema 2

Verifique a ortogonalidade discreta da base trigonométrica, ou seja, 

$$
\sum_{i=0}^Q \phi_n(x_i)\cdot \phi_k(x_i)=0 \qquad k\ne n, N\: \text{não é múltiplo de}\: Q/2
$$

para 

\begin{align}
\phi_0(x) &= \frac{1}{2}\\
\phi_n(x) &= \cos\left(\frac{2\pi n x}{L_0}\right) \qquad 0\le n \le N \\
\phi_{n+N}(x) &= \sin\left(\frac{2\pi n x}{L_0}\right) \qquad 1\le n \le N 
\end{align}



In [ ]:
Q = 10
x = qpoints(Q)

In [ ]:
L0 = 2.0
n = 3; k =12
sum(sin.((2π*n/L0)*x) .* cos.((2π*k/L0)*x))

## Problema 3

Nos exemplos iniciais usando a DFT, ao se escolher os pontos, foi utilizado o seguinte código:
```julia
L₀ = 1.0; Q = 16; freq=1
x = range(0.0, L₀, length=Q+1)[1:Q]
```
O que acontece se utilzarmos a expressão mais simples a seguir?
```julia
x = range(0.0, L₀, length=Q)
```
O que está acontecendo? Você consegue explicar?

In [ ]:
L₀ = 1.0; Q = 16; freq=1; 
x = range(0.0, L₀, length=Q+1)[1:Q]
y = sin.(freq.*2π.*x); f = (0:Q-1) / L₀
Y = 1/Q * fft(y);
plot(f, imag.(Y), "ro")
plot(f, real.(Y), "xb")

In [ ]:
L₀ = 1.0; Q = 16; freq=1; 
x = range(0.0, L₀, length=Q)
y = sin.(freq.*2π.*x); f = (0:Q-1) / L₀
Y = 1/Q * fft(y);
plot(f, imag.(Y), "ro")
plot(f, real.(Y), "xb")


**O que está acontecendo?**

O que a gente quer?  Interpolar a função $\sin2\pi x$ para $0\le x\le 1$ 

Visualmente o que ocorre pode ser visto no gráfico a seguir:


In [ ]:
Q = 4; x1 = range(0, 1, length=Q+1)[1:Q]; dx1 = x1[2]-x1[1];
x2 = range(0, 1, length=Q); dx2 = x2[2]-x2[1]
subplot(121); plot(x1, zeros(Q), "r.")
for i in 1:Q
    plot([x1[i], x1[i]+dx1], [0, 0], "k-")
end
subplot(122); plot(x2, zeros(Q), "r.")
for i in 1:Q
    plot([x2[i], x2[i]+dx2], [0, 0], "k-")
end

Temos uma inconsistência!

Como só passamos o número de pontos, na verdade estamos aproximando a função mais um trecho reto:

In [ ]:
Q = 4; x1 = range(0, 1, length=Q+1)[1:Q]; dx1 = x1[2]-x1[1]
dx1 = 1.0 / Q; xx1 = 0:0.01:1; yy1 = sin.(2π.*xx1)
xx = [xx1; 1.0 + dx1]; yy = [yy1; 0.0]
plot(xx, yy)

## Problema 4

Implemente a DFT usando a definição. Verifique como aumenta o custo computacional quando o número de pontos aumenta. 

Como foi mostrado, esta operação é uma multiplicação de matriz por vetor. Verifique se invertendo a ordem dos loops afeta o desempenho. 

### DFT - Discrete Fourier Transform (transformada de Fourier discreta)

$$
c_n = X_n = \frac{1}{N}\sum_{k=0}^{N-1} x_k \exp\left(-\frac{2\pi n k}{N} \right)
$$

Chamando 
$$
w_k = w^k = \exp\left(-\frac{2\pi k}{N}\right)
$$
Repare que isto pode ser escrito como uma multiplicação de matrizes:

$$
\left(\begin{matrix} X_0 \\ X_1 \\ \vdots \\ X_{N-1}\end{matrix} \right)
= 
\left(\begin{matrix} 1 & 1 & \cdots & 1\\
1 & w & \cdots & w^{N-1} \\
\vdots & \vdots & \ddots &\vdots \\
1 & w^N & \cdots & w^{(N-1)^2}\\
\end{matrix}\right) 
\cdot 
\left(\begin{matrix} x_0 \\ x_1 \\ \vdots \\ x_{N-1}\end{matrix} \right) 
$$




In [ ]:
function mydft(x)
    N = length(x)
    X = zeros(ComplexF64,N)
    w = exp(-2π*im/N)

    for i in 1:N
        for k in 1:N
            X[i] += x[k] * w^((i-1)*(k-1))
        end
    end
    
    return X
end
            


In [ ]:
x = rand(10)
X1 = mydft(x)
X2 = fft(x)
X1 - X2

In [ ]:
function mydft!(x, X)
    N = length(x)
    w = exp(-2π*im/N)
    for i in 1:N
        for k in 1:N
            X[i] += x[k] * w^((i-1)*(k-1))
        end
    end
    return X
end
   
mydft(x) = mydft!(x, zeros(ComplexF64, length(x)))


## Problema 5

(Desafio) procure na net ou em algum livro como implementar a FFT. Tente!

In [ ]:
"""
Retorna a potência de dois de um número. 
-1 se não for uma potência de 2
"""
function twopower(n::Int)
    l2 = log2(n)
    if round(l2) != l2
        return -1
    end
    return Int(l2)
end


In [ ]:
function myrfft(x)
    Q = length(x)
    p = twopower(n)
    if p < -1
        error("Só funciona com arrays com numero de elementos que é potência de 2")
    end
    
    N = Q÷2 + 1
    X = zeros(ComplexF64, N)
    
    for k in 0:N-1
        Xk = zero(ComplexF64)
       
    end
end

## Problema 6

A transformada de Fourier está diretamente relacionada com a série de Fourier e é dada por:

$$
X(\omega) = \frac{1}{2\pi} \int_{-\infty}^\infty x(t)e^{-i\omega t}\: dt
$$

 ### 6.1 Calcula a transformada de Fourier para a função 
  $$
  x(t) = \left\{\begin{matrix} 1 & -1 < x < 1 \\ 0 & x \ge 1\end{matrix}\right.
  $$
 
Solução:
$$
X(\omega) = \frac{1}{2\pi}\int_{-\infty}^\infty x(t)e^{-i\omega t}\: dt = 
\frac{1}{2\pi}\int_{-1}^1 e^{-i\omega t}\: dt = \frac{1}{2\pi}\left.\frac{i e^{-i\omega}}{\omega}\right|_{-1}^1 = 
\frac{\sin\omega}{\pi\omega}
$$

Função sinc:
$$
\text{sinc} = \frac{\sin\pi x}{\pi x}
$$

In [ ]:
x = -15:0.01:15
y = 1/π .* sinc.(x./π)
plot(x,y)

### 6.2 Discretize a função e use a FFT para calcular a transformada.

$$
X(\omega) = \frac{1}{2\pi}\int_{-\infty}^\infty x(t)e^{-i\omega t}\: dt = 
\frac{1}{2\pi}\int_{-T_0/2}^{T_0/2} e^{-i\omega t}\: dt \approx \frac{1}{2\pi} \sum_{k=0}^{N-1} x(t_k) e^{-i\omega t_k}\Delta t \qquad \Delta t = \frac{T_0}{N} \qquad t_k = -\frac{T_0}{2} + k\cdot \Delta t
$$

Ou seja, 

$$
X(\omega) \approx \frac{1}{2\pi} \sum_{k=0}^{N-1} x(t_k) e^{-i\omega t_k}\Delta t = 
\frac{\Delta t}{2\pi} \sum_{k=0}^{N-1} x_k e^{-i\omega(-T_0/2 + k\Delta t)} = 
\frac{\Delta t}{2\pi}\cdot e^{\frac{i\omega T_0}{2}}\cdot \sum_{k=0}^{N-1} x_k e^{-i\omega k\Delta t} = 
$$

Discretizando $\omega = \omega_j = j\cdot \omega_0 = \frac{2\pi j}{T_0}$, $0 \le j 

$$
X_j = X(\omega_j) = \frac{\Delta t}{2\pi}\cdot e^{\frac{2\pi j T_0}{2{T_0}}}\cdot \sum_{k=0}^{N-1} x_k e^{-\frac{2\pi i j\omega k\Delta t}{N\cdot \Delta t}} = 
 \frac{\Delta t}{2\pi}\cdot e^{\pi i j}\cdot \sum_{k=0}^{N-1} x_k \exp\left(-\frac{2\pi i j k}{N}\right)
$$
ou seja
$$
X_j = X(\omega_j) =
\frac{\Delta t}{2\pi}\cdot e^{\pi i j}\cdot \sum_{i=0}^{N-1} x_k \exp\left(-\frac{2\pi i j k}{N}\right)
$$
A somatória no ultimo termo é a DFT. O termo $e^{\pi i j}$ é corresponde à translação.

In [ ]:
Q = 32
Δt = 2.0 / Q
x = ones(Q)
X = rfft(x)
N = length(X)
for j in 0:N-1
    X[j+1] = X[j+1] * Δt/(2π) * exp(π*j*im) 
end
freq = rfftfreq(Q, 1/Δt)
w = 2π*freq

In [ ]:
ww = range(0, maximum(w), length=1001)
yy = 1/π .* sinc.(ww./π)
plot(ww./(2π), yy)
plot(freq, real.(X), "ro")

### 6.3 Tente verificar o que acontece se você adiciona zeros à esquerda e à direita da função discretizada.
 


In [ ]:
Q1 = 32
nn = 5
x = [zeros(nn*Q1); ones(Q1); zeros(nn*Q1-1)]
Q = length(x)
Δt = 2.0 / Q1
X = rfft(x)
N = length(X)
for j in 0:N-1
    X[j+1] = X[j+1] * Δt/(2π) * exp(π*j*im) 
end
freq = rfftfreq(Q, 1/Δt)
w = 2π*freq;

In [ ]:
plot(freq, real.(X))
plot(freq, real.(X), "r.")


## Problema 7

Faça alguns exemplos do pacote `ApproxFun` (<https://github.com/JuliaApproximation/ApproxFun.jl>)
